## Loading Fiducial data at each waypoint

In [54]:
import sys
import numpy as np
import pickle

# Add ROS packages to the path
sys.path.append('/com.docker.devenvironments.code/catkin_ws/devel/lib/python3/dist-packages')
sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/')

import fiducial

indoor = False
raw_data = False

test_data_path = '/com.docker.devenvironments.code/test_data/'

# Set the path to the data for all 3 walks
fid_path_indoor = [
    test_data_path + 'walk2/fid-walk2.pickle',
    test_data_path + 'walk3/fid-walk3.pickle',
    test_data_path + 'walk4/fid-walk4.pickle'
]

way_fid_path_indoor = [
    test_data_path + '27Mar/walk2/way-walk2.pickle',
    test_data_path + '27Mar/walk3/way-walk3.pickle',
    test_data_path + '27Mar/walk4/way-walk4.pickle'
]

fid_path_outdoor = [
    test_data_path + '28Mar/fid1.pickle',
    test_data_path + '28Mar/fid2.pickle',
    test_data_path + '28Mar/fid3.pickle'
]

way_fid_path_outdoor = [
    test_data_path + '28Mar/way1.pickle',
    test_data_path + '28Mar/way2.pickle',
    test_data_path + '28Mar/way3.pickle'
]

if indoor:
    fid_path = fid_path_indoor
    way_fid_path = way_fid_path_indoor
    way_fid_path_processed = "indoor_way_fid_data.pickle"
    output_path = "variation_indoor.txt"

else:
    fid_path = fid_path_outdoor
    way_fid_path = way_fid_path_outdoor
    way_fid_path_processed = "outdoor_way_fid_data.pickle"
    output_path = "variation_outdoor.txt"

if raw_data:
    # Load raw data: Depractated; since they do not always match the same reference frame_id
    way_fid_data = []
    for p in way_fid_path:
        with open(p, 'rb') as f:
            way_fid_data.append(pickle.load(f))

else:
    # Load processed data
    with open(way_fid_path_processed, 'rb') as f:
        way_fid_data = pickle.load(f)

## Fiducial aggregate statistics

In [55]:
waypoints = list(way_fid_data[0].keys())
n = len(way_fid_data)

# For each waypoint, get the fiducial data from each walk
fiducial_by_waypoint = {}
for w in waypoints:
    fiducial_by_waypoint[w] = []
    for i in range(n):
        fiducial_by_waypoint[w].append(way_fid_data[i][w][0])

# For each waypoint, get the min, max, average of the fiducial data
total_reference_frames = set()
fiducial_variance = {}
all_detected_tag_ids = set()
for w in waypoints:
    detected_fiducials_at_this_waypoint = []
    detected_tag_ids = set()
    for i in range(n):
        detected_this_run = {f.tag_id: f for f in fiducial_by_waypoint[w][i]}
        for tag_id in detected_this_run:
            detected_tag_ids.add(tag_id)
            all_detected_tag_ids.add(tag_id)
        detected_fiducials_at_this_waypoint.append(detected_this_run)

    # Calculate the min, max, average, variance for each tag_id
    fiducial_statistics_at_this_waypoint = {}
    for tag_id in detected_tag_ids:
        detections_for_this_fiducial = []
        for i in range(n):
            if tag_id not in detected_fiducials_at_this_waypoint[i]:
                continue

            detections_for_this_fiducial.append(detected_fiducials_at_this_waypoint[i][tag_id])

        if len(detections_for_this_fiducial) < 3:
            continue

        # Calculate the min, max, average, variance, var_from_cov for this tag_id
        fiducial_statistics_at_this_waypoint[tag_id] = {}
        min_x = min([f.fiducial_pose.pose.position.x for f in detections_for_this_fiducial])
        max_x = max([f.fiducial_pose.pose.position.x for f in detections_for_this_fiducial])
        avg_x = np.mean([f.fiducial_pose.pose.position.x for f in detections_for_this_fiducial])
        var_x = np.var([f.fiducial_pose.pose.position.x for f in detections_for_this_fiducial])
        avg_var_from_cov_x = np.mean([f.pose_covariance.covariance[0] for f in detections_for_this_fiducial])
        fiducial_statistics_at_this_waypoint[tag_id]['x'] = (min_x, max_x, avg_x, var_x, avg_var_from_cov_x)

        min_y = min([f.fiducial_pose.pose.position.y for f in detections_for_this_fiducial])
        max_y = max([f.fiducial_pose.pose.position.y for f in detections_for_this_fiducial])
        avg_y = np.mean([f.fiducial_pose.pose.position.y for f in detections_for_this_fiducial])
        var_y = np.var([f.fiducial_pose.pose.position.y for f in detections_for_this_fiducial])
        avg_var_from_cov_y = np.mean([f.pose_covariance.covariance[7] for f in detections_for_this_fiducial])
        fiducial_statistics_at_this_waypoint[tag_id]['y'] = (min_y, max_y, avg_y, var_y, avg_var_from_cov_y)

        min_z = min([f.fiducial_pose.pose.position.z for f in detections_for_this_fiducial])
        max_z = max([f.fiducial_pose.pose.position.z for f in detections_for_this_fiducial])
        avg_z = np.mean([f.fiducial_pose.pose.position.z for f in detections_for_this_fiducial])
        var_z = np.var([f.fiducial_pose.pose.position.z for f in detections_for_this_fiducial])
        avg_var_from_cov_z = np.mean([f.pose_covariance.covariance[14] for f in detections_for_this_fiducial])
        fiducial_statistics_at_this_waypoint[tag_id]['z'] = (min_z, max_z, avg_z, var_z, avg_var_from_cov_z)

        ref_frame = [f.fiducial_pose.header.frame_id for f in detections_for_this_fiducial]
        _ = [total_reference_frames.add(i) for i in ref_frame]
        fiducial_statistics_at_this_waypoint[tag_id]['ref_frame'] = ref_frame

        num_detections = len(detections_for_this_fiducial)
        fiducial_statistics_at_this_waypoint[tag_id]['num_detections'] = num_detections
        fiducial_statistics_at_this_waypoint[tag_id]['data_x'] = [f.fiducial_pose.pose.position.x for f in detections_for_this_fiducial]

        # Checking specific tag_id for debugging
        if tag_id == 403:
            print("Waypoint: {}".format(w))
            print("Tag ID: {}".format(tag_id))
            print("Num Detections: {}".format(fiducial_statistics_at_this_waypoint[tag_id]['data_x']))

    # Save the statistics for this waypoint
    fiducial_variance[w] = fiducial_statistics_at_this_waypoint

print("Used reference frames:")
print(total_reference_frames)

# Calculate the variance of each detection at each waypoint over runs, using max-min
variance_for_each_detection = {}
for w in waypoints:
    variance_for_each_detection[w] = {}
    for tag_id in fiducial_variance[w]:
        variance_for_each_detection[w][tag_id] = {}
        for axis in ['x', 'y', 'z']:
            min_val, max_val, avg_val, var_val, avg_var_from_cov = fiducial_variance[w][tag_id][axis]
            variance_for_each_detection[w][tag_id][axis] = (max_val - min_val)
    
# Calculate the Mean Variation (mv) over all waypoints over all detections, one for each axis, filtering out NaN values before averaging
mv = {}
for axis in ['x', 'y', 'z']:
    mv[axis] = np.nanmean([variance_for_each_detection[w][tag_id][axis] for w in waypoints for tag_id in variance_for_each_detection[w]])

# Create a map of waypoint to position in sequence using waypoints variable
waypoint_to_position = {}
for i in range(len(waypoints)):
    waypoint_to_position[waypoints[i]] = i

# Save the variation for each axis in a .txt file for loading to MATLAB later
"""
waypoint tag_id x y z
1 350 0.1 0.2 0.3
2 351 0.2 0.3 0.4
"""
with open(output_path, "w") as f:
    # Write the header
    line_to_write = "waypoint tag_id x y z\n"
    
    # Write the variation for each waypoint for each axis
    for w in waypoints:
        for tag_id in fiducial_variance[w]:
            line_to_write += f"{waypoint_to_position[w]} {tag_id} "
            for axis in ['x', 'y', 'z']:
                line_to_write += f"{variance_for_each_detection[w][tag_id][axis]} "
            line_to_write += "\n"
    
    f.write(line_to_write)

print(f"Number of unique tag_ids detected: {len(all_detected_tag_ids)}")
print(f"Number of waypoints: {len(waypoints)}")
print(f"Number of runs: {n}")
print(f"Number of detections: {sum([len(fiducial_variance[w]) for w in waypoints])}")

print(f"MV x: {mv['x']}")
print(f"MV y: {mv['y']}")
print(f"MV z: {mv['z']}")

Used reference frames:
{'body'}
Number of unique tag_ids detected: 10
Number of waypoints: 147
Number of runs: 3
Number of detections: 44
MV x: 0.23609790087539775
MV y: 0.07086109521432465
MV z: 0.02531005489707351


In [56]:
# Calculate the average variance for all fiducials in x, y, z
fiducial_variance_avg = {}
for w in waypoints:
    fiducial_variance_avg[w] = {}
    for tag_id in fiducial_variance[w]:
        fiducial_variance_avg[w][tag_id] = {}
        for axis in ['x', 'y', 'z']:
            fiducial_variance_avg[w][tag_id][axis] = fiducial_variance[w][tag_id][axis][3]

# Aggregate the average variance for all fiducials and all waypoints in x, y, z
fiducial_variance_avg_all = {}
for axis in ['x', 'y', 'z']:
    fiducial_variance_avg_all[axis] = []
    for w in waypoints:
        for tag_id in fiducial_variance_avg[w]:
            fiducial_variance_avg_all[axis].append(fiducial_variance_avg[w][tag_id][axis])
    
    fiducial_variance_avg_all[axis] = np.mean(fiducial_variance_avg_all[axis])

print(f"Average variance in x: {fiducial_variance_avg_all['x']}")
print(f"Average variance in y: {fiducial_variance_avg_all['y']}")
print(f"Average variance in z: {fiducial_variance_avg_all['z']}")

"""
Repeat the calculation using variance from covariance matrix
"""
# Calculate the average variance for all fiducials in x, y, z
fiducial_variance_avg = {}
for w in waypoints:
    fiducial_variance_avg[w] = {}
    for tag_id in fiducial_variance[w]:
        fiducial_variance_avg[w][tag_id] = {}
        for axis in ['x', 'y', 'z']:
            fiducial_variance_avg[w][tag_id][axis] = fiducial_variance[w][tag_id][axis][4]

# Aggregate the average variance for all fiducials and all waypoints in x, y, z
fiducial_variance_avg_all = {}
for axis in ['x', 'y', 'z']:
    fiducial_variance_avg_all[axis] = []
    for w in waypoints:
        for tag_id in fiducial_variance_avg[w]:
            fiducial_variance_avg_all[axis].append(fiducial_variance_avg[w][tag_id][axis])
    
    fiducial_variance_avg_all[axis] = np.mean(fiducial_variance_avg_all[axis])

print(f"Average variance from cov in x: {fiducial_variance_avg_all['x']}")
print(f"Average variance from cov in y: {fiducial_variance_avg_all['y']}")
print(f"Average variance from cov in z: {fiducial_variance_avg_all['z']}")

Average variance in x: 0.03239900935614418
Average variance in y: 0.0015127028361570163
Average variance in z: 0.00014675465045740544
Average variance from cov in x: 0.01194924811717108
Average variance from cov in y: 0.005410254965018708
Average variance from cov in z: 0.0029201144134491986


## Reviewing size of dataset

In [40]:
# Calculate total number of fiducial detections
total_fiducial_detections = []
for w in waypoints:
    for tag_id in fiducial_variance[w]:
        total_fiducial_detections.append(tag_id)
num_tag_ids_unique = len(set(total_fiducial_detections))

print(f"Total number of fiducial detections: {len(total_fiducial_detections)}")

# Print number of waypoints
print(f"Number of waypoints: {len(waypoints)}")

# Print number of fiducial tag_ids
print(f"Number of fiducial tag_ids: {num_tag_ids_unique}")

# Print number of runs
print(f"Number of runs: {n}")


Total number of fiducial detections: 107
Number of waypoints: 101
Number of fiducial tag_ids: 17
Number of runs: 3


## Save filtered position of Fiducials in vision frame

In [13]:
# Extract the filtered_fiducial_pose for each fiducial tag_id
filtered_fiducial_poses = {}
for w in waypoints:
    # Use fiducial_by_waypoint
    for fiducials in fiducial_by_waypoint[w]:
        if not fiducials:
            continue

        for f in fiducials:
            if f.tag_id not in filtered_fiducial_poses:
                filtered_fiducial_poses[f.tag_id] = []
            filtered_fiducial_poses[f.tag_id].append(f.filtered_fiducial_pose)

# Calculate the variance for each fiducials in x, y, z
filtered_fiducial_variance = {}
for tag_id in filtered_fiducial_poses:
    filtered_fiducial_variance[tag_id] = {}
    for axis in ['x', 'y', 'z']:
        filtered_fiducial_variance[tag_id][axis] = np.var([getattr(f.pose.position, axis) for f in filtered_fiducial_poses[tag_id]])

print(filtered_fiducial_variance)

# Aggregate the average variance for all fiducials and all waypoints in x, y, z
filtered_fiducial_variance_avg_all = {}
for axis in ['x', 'y', 'z']:
    filtered_fiducial_variance_avg_all[axis] = []
    for tag_id in filtered_fiducial_variance:
        filtered_fiducial_variance_avg_all[axis].append(filtered_fiducial_variance[tag_id][axis])
    
    filtered_fiducial_variance_avg_all[axis] = np.mean(filtered_fiducial_variance_avg_all[axis])

print(f"Average filtered_fiducial_pose in x: {filtered_fiducial_variance_avg_all['x']}")
print(f"Average filtered_fiducial_pose in y: {filtered_fiducial_variance_avg_all['y']}")
print(f"Average filtered_fiducial_pose in z: {filtered_fiducial_variance_avg_all['z']}")

{350: {'x': 3.812757519415395, 'y': 11.09577853213959, 'z': 0.0010274956201705798}, 400: {'x': 2.51247578395849, 'y': 7.2569277193744615, 'z': 0.0009554110259667335}, 401: {'x': 1.3613243190982596, 'y': 3.0523851164195586, 'z': 0.000745318333230216}, 402: {'x': 0.7084369584691166, 'y': 0.05840223177596858, 'z': 0.0011036998334070472}, 403: {'x': 0.16529585666701388, 'y': 3.432637067405014, 'z': 0.0005410256382088529}, 409: {'x': 0.45599253435286535, 'y': 5.173808927614945, 'z': 4.295276245751007e-05}, 404: {'x': 0.09305284228204913, 'y': 11.337228011039711, 'z': 0.0003993457019513893}, 405: {'x': 0.43811936038727445, 'y': 18.776764430868663, 'z': 0.00045907740720042774}, 406: {'x': 6.758181658016805, 'y': 65.12350148328903, 'z': 3.801395746543977e-05}, 407: {'x': 6.811831189137742, 'y': 32.215200596804806, 'z': 1.4597169182112392e-05}, 408: {'x': 2.078903552318673, 'y': 14.927959848612138, 'z': 4.946709373693728e-05}, 410: {'x': 0.09705794481167261, 'y': 1.164255550784809, 'z': 6.17484